## Content


- **Grouping**
    - Split, Apply, Combine
    - `groupby()`
- **Group based Aggregates**

- **Group based Filtering**

- **Group based Apply**
    - `apply()`




## Importing Data

Let's first import our data and prepare it as we did in the last lecture

In [1]:
import pandas as pd
import numpy as np
# !gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
# !gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
movies = pd.read_csv('movies.csv', index_col=0)
directors = pd.read_csv('directors.csv',index_col=0)
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')  
data.drop(['director_id','id_y'],axis=1,inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'movies.csv'

## Grouping



#### How can we know the number of movies released by a particular director, say, Christopher Nolan?

In [ ]:
data.loc[data['director_name'] == 'Christopher Nolan',['title']].count()

title    8
dtype: int64

#### What if we have to do find number of movies of each director? 

We have value_counts() for this

In [ ]:
data["director_name"].value_counts()

Steven Spielberg      26
Martin Scorsese       19
Clint Eastwood        19
Woody Allen           18
Ridley Scott          16
                      ..
Tim Hill               5
Jonathan Liebesman     5
Roman Polanski         5
Larry Charles          5
Nicole Holofcener      5
Name: director_name, Length: 199, dtype: int64

How does this exactly work? 

We can assume pandas must have **grouped the rows internally** to find the count

But what if we need to find some **other metric** besides count?

For example, **average popularity** of each director, or **max rating** among all movies by a director?

#### How can you find the average popularity of each director?

We will have to some group our rows director wise.


#### What is Grouping ?

Simply it could be understood through the terms - Split, apply, combine

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781491912126/files/assets/pyds_03in01.png" height = 350/>

1. **Split**: **Breaking up and grouping** a DataFrame depending on the value of the specified key.

2. **Apply**: Computing **some functio**n, usually an **aggregate, transformation, or filtering**, within the individual groups.

3. **Combine**: **Merge the results** of these operations into an output array.

Note: 

All these steps are to understand the topic

## Group based Aggregates


#### Now, how can we group our data director-wise?


In [ ]:
data.groupby('director_name')

Notice,
- It's a **DataFrameGroupBy type object**

- **NOT a DataFrame** type object

#### What is `groupby('director_name')` doing?

**Grouping all rows** in which **director_name** value is **same**

But it's returning an object, we would want to get information out of this object.

Let's look at few attributes of the same.





#### How can we know the number of groups our data is divided into?

In [ ]:
data.groupby('director_name').ngroups

199

<!-- Access group and corresponding row keys -->

Based on this grouping, how can we find which keys belong to which group?

In [ ]:
data.groupby('director_name').groups

{'Adam McKay': [176, 323, 366, 505, 839, 916], 'Adam Shankman': [265, 300, 350, 404, 458, 843, 999, 1231], 'Alejandro González Iñárritu': [106, 749, 1015, 1034, 1077, 1405], 'Alex Proyas': [95, 159, 514, 671, 873], 'Alexander Payne': [793, 1006, 1101, 1211, 1281], 'Andrew Adamson': [11, 43, 328, 501, 947], 'Andrew Niccol': [533, 603, 701, 722, 1439], 'Andrzej Bartkowiak': [349, 549, 754, 911, 924], 'Andy Fickman': [517, 681, 909, 926, 973, 1023], 'Andy Tennant': [314, 320, 464, 593, 676, 885], 'Ang Lee': [99, 134, 748, 840, 1089, 1110, 1132, 1184], 'Anne Fletcher': [610, 650, 736, 789, 1206], 'Antoine Fuqua': [310, 338, 424, 467, 576, 808, 818, 1105], 'Atom Egoyan': [946, 1128, 1164, 1194, 1347, 1416], 'Barry Levinson': [313, 319, 471, 594, 878, 898, 1013, 1037, 1082, 1143, 1185, 1345, 1378], 'Barry Sonnenfeld': [13, 48, 90, 205, 591, 778, 783], 'Ben Stiller': [209, 212, 547, 562, 850], 'Bill Condon': [102, 307, 902, 1233, 1381], 'Bobby Farrelly': [352, 356, 481, 498, 624, 630, 654, 80

<!-- Access data for specific group -->

#### Now what if we want to extract data of a particular group from this list?

In [ ]:
data.groupby('director_name').get_group('Alexander Payne')

id_x    budget  popularity    revenue            title  vote_average  \
793   45163  30000000          19  105834556    About Schmidt           6.7   
1006  45699  20000000          40  177243185  The Descendants           6.7   
1101  46004  16000000          23  109502303         Sideways           6.9   
1211  46446  12000000          29   17654912         Nebraska           7.4   
1281  46813         0          13          0         Election           6.7   

      vote_count  year month       day    director_name gender  
793          362  2002   Dec    Friday  Alexander Payne    NaN  
1006         934  2011   Sep    Friday  Alexander Payne    NaN  
1101         478  2004   Oct    Friday  Alexander Payne    NaN  
1211         636  2013   Sep  Saturday  Alexander Payne    NaN  
1281         270  1999   Apr    Friday  Alexander Payne    NaN

Great! We are able to extract the data from our DataFrameGroupBy object

But can we extend this to finding an aggregate metric of the data?
#### How can we find count of each director?

This does give us the max value of the data, but for **all the features**

In [ ]:
data.groupby('director_name')['title'].count()

director_name
Adam McKay                      6
Adam Shankman                   8
Alejandro González Iñárritu     6
Alex Proyas                     5
Alexander Payne                 5
                               ..
Wes Craven                     10
Wolfgang Petersen               7
Woody Allen                    18
Zack Snyder                     7
Zhang Yimou                     6
Name: title, Length: 199, dtype: int64

Now say we want to know two aggregations for any feature.

For e.g., the very first year and the latest year a director released a movie

This is basically the `min` and `max` of `year` column, grouped by director

#### How can we find multiple aggregations of any feature?

In [ ]:
data.groupby(['director_name'])["year"].aggregate(['min', 'max'])

min   max
director_name                          
Adam McKay                   2004  2015
Adam Shankman                2001  2012
Alejandro González Iñárritu  2000  2015
Alex Proyas                  1994  2016
Alexander Payne              1999  2013
...                           ...   ...
Wes Craven                   1984  2011
Wolfgang Petersen            1981  2006
Woody Allen                  1977  2013
Zack Snyder                  2004  2016
Zhang Yimou                  2002  2014

[199 rows x 2 columns]

## Group based Filtering



#### How we find details of the movies by high budget directors?


Lets assume, 
- high budget director -> any director with **atleast one movie with budget >100M**  

  
<!-- Notice, the question is **not asking us to give the name of the directors who have directed high budget movies** 

Lets first quickly see, how we would  have solved that 
To begin with, 

#### How can we find the names of high budget directors? -->

We can get the highest budget movie data of every director

<!-- We can,

1. `group` the data by director
2. use `max` of the budget as aggregator -->

In [ ]:
data_dir_budget = data.groupby("director_name")["budget"].max().reset_index()
data_dir_budget.head()

director_name     budget
0                   Adam McKay  100000000
1                Adam Shankman   80000000
2  Alejandro González Iñárritu  135000000
3                  Alex Proyas  140000000
4              Alexander Payne   30000000

#### How can we **filter** out the director names with **max budget >100M**?

In [ ]:
names = data_dir_budget.loc[data_dir_budget["budget"] >= 100, "director_name"]

#### Finally, how can we filter out the details of the movies by these directors?

In [ ]:
data.loc[data['director_name'].isin(names)]

id_x     budget  popularity     revenue  \
0     43597  237000000         150  2787965087   
1     43598  300000000         139   961000000   
2     43599  245000000         107   880674609   
3     43600  250000000         112  1084939099   
4     43602  258000000         115   890871626   
...     ...        ...         ...         ...   
1460  48363          0           3      321952   
1461  48370      27000          19     3151130   
1462  48375          0           7           0   
1463  48376          0           3           0   
1464  48395     220000          14     2040920   

                                         title  vote_average  vote_count  \
0                                       Avatar           7.2       11800   
1     Pirates of the Caribbean: At World's End           6.9        4500   
2                                      Spectre           6.3        4466   
3                        The Dark Knight Rises           7.6        9106   
4                                 Spider-Man 3           5.9        3576   
...                                        ...           ...         ...   
1460                            The Last Waltz           7.9          64   
1461                                    Clerks           7.4         755   
1462                                   Rampage           6.0         131   
1463                                   Slacker           6.4          77   
1464                               El Mariachi           6.6         238   

      year month       day      director_name gender  
0     2009   Dec  Thursday      James Cameron   Male  
1     2007   May  Saturday     Gore Verbinski   Male  
2     2015   Oct    Monday         Sam Mendes   Male  
3     2012   Jul    Monday  Christopher Nolan   Male  
4     2007   May   Tuesday          Sam Raimi   Male  
...    ...   ...       ...                ...    ...  
1460  1978   May    Monday    Martin Scorsese   Male  
1461  1994   Sep   Tuesday        Kevin Smith   Male  
1462  2009   Aug    Friday           Uwe Boll   Male  
1463  1990   Jul    Friday  Richard Linklater   Male  
1464  1992   Sep    Friday   Robert Rodriguez    NaN  

[1465 rows x 12 columns]

Recall `isin()` from last lecture

#### Can we do filtering of groups in a single go?

**YES**

In [ ]:
data.groupby('director_name').filter(lambda x: x["budget"].max() >= 100)

id_x     budget  popularity     revenue  \
0     43597  237000000         150  2787965087   
1     43598  300000000         139   961000000   
2     43599  245000000         107   880674609   
3     43600  250000000         112  1084939099   
4     43602  258000000         115   890871626   
...     ...        ...         ...         ...   
1460  48363          0           3      321952   
1461  48370      27000          19     3151130   
1462  48375          0           7           0   
1463  48376          0           3           0   
1464  48395     220000          14     2040920   

                                         title  vote_average  vote_count  \
0                                       Avatar           7.2       11800   
1     Pirates of the Caribbean: At World's End           6.9        4500   
2                                      Spectre           6.3        4466   
3                        The Dark Knight Rises           7.6        9106   
4                                 Spider-Man 3           5.9        3576   
...                                        ...           ...         ...   
1460                            The Last Waltz           7.9          64   
1461                                    Clerks           7.4         755   
1462                                   Rampage           6.0         131   
1463                                   Slacker           6.4          77   
1464                               El Mariachi           6.6         238   

      year month       day      director_name gender  
0     2009   Dec  Thursday      James Cameron   Male  
1     2007   May  Saturday     Gore Verbinski   Male  
2     2015   Oct    Monday         Sam Mendes   Male  
3     2012   Jul    Monday  Christopher Nolan   Male  
4     2007   May   Tuesday          Sam Raimi   Male  
...    ...   ...       ...                ...    ...  
1460  1978   May    Monday    Martin Scorsese   Male  
1461  1994   Sep   Tuesday        Kevin Smith   Male  
1462  2009   Aug    Friday           Uwe Boll   Male  
1463  1990   Jul    Friday  Richard Linklater   Male  
1464  1992   Sep    Friday   Robert Rodriguez    NaN  

[1465 rows x 12 columns]

Notice what's happening here?
- We first group data by director and then use `groupby().filter` function
- **Groups are filtered if they do not satisfy the boolean criterion** specified by function
- This is called  **Group Based Filtering**



**NOTE**

We are filtering the **groups** here and **not the rows**

==> The result is **not a groupby object** but **regular pandas DataFrame** with the **filtered groups eliminated**

## Group based Apply


<!-- 
#### What if we want to do the transformation of a column using some column's agrregate

Lets say, we want to filter the movies whose budget was even higher than the average revenue of the director from his other movies -->

Now let's assume, we call a movi risky if,
- its budget is higher than the average revenue of its director 

#### How do we filter risky movies? 

We can subtract the average `revenue` of a director from `budget` col, for each director

#### Can we use `apply` here?

Yes!

#### How do we use apply for this column?

- We will define a function to compute the subtraction
- Pass this function in `apply`

<!-- Can use `transform` here?

**No**, since it uses only one column

#### Can you recall **another function like transform** which works with multiple columns? -->

In [ ]:
def func(x):
  x["risky"] = x["budget"] - x["revenue"].mean() >= 0
  return x
data_risky = data.groupby("director_name").apply(func)
data_risky

id_x     budget  popularity     revenue  \
0     43597  237000000         150  2787965087   
1     43598  300000000         139   961000000   
2     43599  245000000         107   880674609   
3     43600  250000000         112  1084939099   
4     43602  258000000         115   890871626   
...     ...        ...         ...         ...   
1460  48363          0           3      321952   
1461  48370      27000          19     3151130   
1462  48375          0           7           0   
1463  48376          0           3           0   
1464  48395     220000          14     2040920   

                                         title  vote_average  vote_count  \
0                                       Avatar           7.2       11800   
1     Pirates of the Caribbean: At World's End           6.9        4500   
2                                      Spectre           6.3        4466   
3                        The Dark Knight Rises           7.6        9106   
4                                 Spider-Man 3           5.9        3576   
...                                        ...           ...         ...   
1460                            The Last Waltz           7.9          64   
1461                                    Clerks           7.4         755   
1462                                   Rampage           6.0         131   
1463                                   Slacker           6.4          77   
1464                               El Mariachi           6.6         238   

      year month       day      director_name gender  risky  
0     2009   Dec  Thursday      James Cameron   Male  False  
1     2007   May  Saturday     Gore Verbinski   Male  False  
2     2015   Oct    Monday         Sam Mendes   Male  False  
3     2012   Jul    Monday  Christopher Nolan   Male  False  
4     2007   May   Tuesday          Sam Raimi   Male  False  
...    ...   ...       ...                ...    ...    ...  
1460  1978   May    Monday    Martin Scorsese   Male  False  
1461  1994   Sep   Tuesday        Kevin Smith   Male  False  
1462  2009   Aug    Friday           Uwe Boll   Male  False  
1463  1990   Jul    Friday  Richard Linklater   Male  False  
1464  1992   Sep    Friday   Robert Rodriguez    NaN  False  

[1465 rows x 13 columns]

Recall `apply()` from our earlier lectures

What did we do here?

- Defined a custom function
- Grouped data acc to `director_name`
- Subtracted mean of `budget` from `revenue` 
- Used apply with the custom function on the grouped data

Lets see if there are any risky movies

In [ ]:
data_risky.loc[data_risky["risky"]]

id_x     budget  popularity     revenue  \
7     43608  200000000         107   586090727   
12    43614  380000000         135  1045713802   
15    43618  200000000          37   310669540   
20    43624  209000000          64   303025485   
24    43630  210000000           3   459359555   
...     ...        ...         ...         ...   
1347  47224    5000000           7     3263585   
1349  47229    5000000           3     4842699   
1351  47233    5000000           6           0   
1356  47263   15000000          10           0   
1383  47453    3500000           4           0   

                                            title  vote_average  vote_count  \
7                               Quantum of Solace           6.1        2965   
12    Pirates of the Caribbean: On Stranger Tides           6.4        4948   
15                                     Robin Hood           6.2        1398   
20                                     Battleship           5.5        2114   
24                          X-Men: The Last Stand           6.3        3525   
...                                           ...           ...         ...   
1347                          The Sweet Hereafter           6.8         103   
1349                         90 Minutes in Heaven           5.4          40   
1351                                Light Sleeper           5.7          15   
1356                           Dying of the Light           4.5         118   
1383                  In the Name of the King III           3.3          19   

      year month        day   director_name gender  risky  
7     2008   Oct   Thursday    Marc Forster   Male   True  
12    2011   May   Saturday    Rob Marshall   Male   True  
15    2010   May  Wednesday    Ridley Scott   Male   True  
20    2012   Apr  Wednesday      Peter Berg   Male   True  
24    2006   May  Wednesday    Brett Ratner   Male   True  
...    ...   ...        ...             ...    ...    ...  
1347  1997   May  Wednesday     Atom Egoyan   Male   True  
1349  2015   Sep     Friday  Michael Polish   Male   True  
1351  1992   Aug     Friday   Paul Schrader    NaN   True  
1356  2014   Dec   Thursday   Paul Schrader    NaN   True  
1383  2013   Dec     Friday        Uwe Boll   Male   True  

[131 rows x 13 columns]

Yes, there are some 131 movies whose budget was **greater than average** earnings of its director

## Multi-Indexing

<!-- After **grouping the data**, select the required column and **apply an aggregate** on it -->

Now, lets say, you want to find who is the **most productive director**

#### Which director according to you would be considered as most productive ?

- Will you decide based on the **number of movies** released by a director?

Or 

- will consider **quality into consideration also?**


Or 

- will you also consider the amount of business the movie is doing? 



To simplify, 

Lets calculate who has directed maximum number of movies

In [ ]:
data.groupby(['director_name'])['title'].count().sort_values(ascending=False)

director_name
Steven Spielberg    26
Clint Eastwood      19
Martin Scorsese     19
Woody Allen         18
Robert Rodriguez    16
                    ..
Paul Weitz           5
John Madden          5
Paul Verhoeven       5
John Whitesell       5
Kevin Reynolds       5
Name: title, Length: 199, dtype: int64

Looks like `Steven Spielberg` has directed maximum number of movies

#### But does it make `Steven` the most productive director?

Chances are, he might be **active for more years** than other directors

#### How would you calculate active years for **every director**?

We can subtract both `min` and `max` of `year`

#### How can we calculate multiple aggregates such as `min` and `max`, along with count of `titles` together?

In [ ]:
data_agg = data.groupby(['director_name'])[["year", "title"]].aggregate({"year":['min','max'], "title": "count"})
data_agg


year       title
                              min   max count
director_name                                
Adam McKay                   2004  2015     6
Adam Shankman                2001  2012     8
Alejandro González Iñárritu  2000  2015     6
Alex Proyas                  1994  2016     5
Alexander Payne              1999  2013     5
...                           ...   ...   ...
Wes Craven                   1984  2011    10
Wolfgang Petersen            1981  2006     7
Woody Allen                  1977  2013    18
Zack Snyder                  2004  2016     7
Zhang Yimou                  2002  2014     6

[199 rows x 3 columns]

Notice,
- `director_name` column has turned into **row labels**
- There are multiple levels for the column names

This is called **Multi-index Dataframe**

#### What is Multi-index Dataframe ?
  - It can have **multiple indexes along a dimension**
    - no of dimensions remain same though => 2D
  - Multi-level indexes are **possible both for rows and columns**


In [ ]:
data_agg.columns #Printing the columns for better clarity

MultiIndex([( 'year',   'min'),
            ( 'year',   'max'),
            ('title', 'count')],
           )

The level-1 column names are `year` and `title`

#### What would happen if we print the col `year` of this multi-index dataframe?

In [ ]:
data_agg["year"]

min   max
director_name                          
Adam McKay                   2004  2015
Adam Shankman                2001  2012
Alejandro González Iñárritu  2000  2015
Alex Proyas                  1994  2016
Alexander Payne              1999  2013
...                           ...   ...
Wes Craven                   1984  2011
Wolfgang Petersen            1981  2006
Woody Allen                  1977  2013
Zack Snyder                  2004  2016
Zhang Yimou                  2002  2014

[199 rows x 2 columns]

#### How can we convert multi-level back to only one level of columns?

Example: `year_min`, `year_max`, `title_count`

In [ ]:
data_agg.columns = ['_'.join(col) for col in data_agg.columns]
data_agg

year_min  year_max  title_count
director_name                                               
Adam McKay                       2004      2015            6
Adam Shankman                    2001      2012            8
Alejandro González Iñárritu      2000      2015            6
Alex Proyas                      1994      2016            5
Alexander Payne                  1999      2013            5
...                               ...       ...          ...
Wes Craven                       1984      2011           10
Wolfgang Petersen                1981      2006            7
Woody Allen                      1977      2013           18
Zack Snyder                      2004      2016            7
Zhang Yimou                      2002      2014            6

[199 rows x 3 columns]

Since these were tuples, we can just join them

In [ ]:
data.groupby('director_name')[['year', 'title']].agg(
    year_max=('year','max'),
    year_min=('year','min'),
    title_count=('title','count')
)


year_max  year_min  title_count
director_name                                               
Adam McKay                       2015      2004            6
Adam Shankman                    2012      2001            8
Alejandro González Iñárritu      2015      2000            6
Alex Proyas                      2016      1994            5
Alexander Payne                  2013      1999            5
...                               ...       ...          ...
Wes Craven                       2011      1984           10
Wolfgang Petersen                2006      1981            7
Woody Allen                      2013      1977           18
Zack Snyder                      2016      2004            7
Zhang Yimou                      2014      2002            6

[199 rows x 3 columns]

Columns look good, but we may want to turn back the row labels into a proper column as well

#### How can we convert row labels into a column?

In [ ]:
data_agg.reset_index()

director_name  year_min  year_max  title_count
0                     Adam McKay      2004      2015            6
1                  Adam Shankman      2001      2012            8
2    Alejandro González Iñárritu      2000      2015            6
3                    Alex Proyas      1994      2016            5
4                Alexander Payne      1999      2013            5
..                           ...       ...       ...          ...
194                   Wes Craven      1984      2011           10
195            Wolfgang Petersen      1981      2006            7
196                  Woody Allen      1977      2013           18
197                  Zack Snyder      2004      2016            7
198                  Zhang Yimou      2002      2014            6

[199 rows x 4 columns]


Recall, 

We learnt `reset_index()` earlier

#### Using the new features, can we find the most productive director?

First calculate how many years the director has been active.



In [ ]:
data_agg["yrs_active"] = data_agg["year_max"] - data_agg["year_min"]
data_agg

year_min  year_max  title_count  yrs_active
director_name                                                           
Adam McKay                       2004      2015            6          11
Adam Shankman                    2001      2012            8          11
Alejandro González Iñárritu      2000      2015            6          15
Alex Proyas                      1994      2016            5          22
Alexander Payne                  1999      2013            5          14
...                               ...       ...          ...         ...
Wes Craven                       1984      2011           10          27
Wolfgang Petersen                1981      2006            7          25
Woody Allen                      1977      2013           18          36
Zack Snyder                      2004      2016            7          12
Zhang Yimou                      2002      2014            6          12

[199 rows x 4 columns]

Then calculate rate of directing movies by `title_count`/`yrs_active`

In [ ]:
data_agg["movie_per_yr"] = data_agg["title_count"] / data_agg["yrs_active"]
data_agg

year_min  year_max  title_count  yrs_active  \
director_name                                                              
Adam McKay                       2004      2015            6          11   
Adam Shankman                    2001      2012            8          11   
Alejandro González Iñárritu      2000      2015            6          15   
Alex Proyas                      1994      2016            5          22   
Alexander Payne                  1999      2013            5          14   
...                               ...       ...          ...         ...   
Wes Craven                       1984      2011           10          27   
Wolfgang Petersen                1981      2006            7          25   
Woody Allen                      1977      2013           18          36   
Zack Snyder                      2004      2016            7          12   
Zhang Yimou                      2002      2014            6          12   

                             movie_per_yr  
director_name                              
Adam McKay                       0.545455  
Adam Shankman                    0.727273  
Alejandro González Iñárritu      0.400000  
Alex Proyas                      0.227273  
Alexander Payne                  0.357143  
...                                   ...  
Wes Craven                       0.370370  
Wolfgang Petersen                0.280000  
Woody Allen                      0.500000  
Zack Snyder                      0.583333  
Zhang Yimou                      0.500000  

[199 rows x 5 columns]

Now finally sort the values

In [ ]:
data_agg.sort_values("movie_per_yr", ascending=False)

year_min  year_max  title_count  yrs_active  movie_per_yr
director_name                                                              
Tyler Perry           2006      2013            9           7      1.285714
Jason Friedberg       2006      2010            5           4      1.250000
Shawn Levy            2002      2014           11          12      0.916667
Robert Rodriguez      1992      2014           16          22      0.727273
Adam Shankman         2001      2012            8          11      0.727273
...                    ...       ...          ...         ...           ...
Lawrence Kasdan       1985      2012            5          27      0.185185
Luc Besson            1985      2014            5          29      0.172414
Robert Redford        1980      2010            5          30      0.166667
Sidney Lumet          1976      2006            5          30      0.166667
Michael Apted         1980      2010            5          30      0.166667

[199 rows x 5 columns]

**Conclusion:**

==> "Tyler Perry" turns out to be the **truly most productive director**
